In [1]:
from google.colab import drive
drive.mount('/content/drive')
# カレントディレクトリの指定
import os
os.chdir('/content/drive/MyDrive/分析コンペ/05_ProbSpace/民泊サービスの宿泊料金予測/')

Mounted at /content/drive


In [ ]:
#!pip install optuna

In [2]:
import datetime
import numpy as np
import pandas as pd
import warnings
#import optuna
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from src.config import *
import src.preprocessing as pr

warnings.filterwarnings('ignore')

In [3]:
df_train = pd.read_csv('input/train_data.csv', parse_dates=[COL_LAST_REVIEW], dtype=DICT_DTYPES)
df_test = pd.read_csv('input/test_data.csv', parse_dates=[COL_LAST_REVIEW], dtype=DICT_DTYPES)
df_train_station_info = pd.read_csv('input/train_data_distance_from_station.csv', dtype=DICT_DTYPES)
df_test_station_info = pd.read_csv('input/test_data_distance_from_station.csv', dtype=DICT_DTYPES)
df_train_svd_name = pd.read_csv('input/train_data_name_features.csv')
df_test_svd_name = pd.read_csv('input/test_data_name_features.csv')
sample_sub = pd.read_csv('input/submission.csv')

In [4]:
LIST_USE_COL = [COL_NEIGHBOURHOOD,
                COL_LATITUDE,
                COL_LONGITUDE,
                COL_ROOM_TYPE,
                COL_MINIMUM_NIGHTS,
                COL_NUMBER_OF_REVIEWS,
                COL_ELAPSED_DAYS,
                COL_REVIEWS_PER_MONTH,
                COL_AVAILABILITY_365]

LIST_LABEL_ENC = [COL_NEIGHBOURHOOD,
                  COL_ROOM_TYPE]

In [5]:
df_all = pd.concat([
    df_train,
    df_test
]).reset_index(drop=True)

In [6]:
# 2020.4.30からの経過日数を特徴量として追加
df_all['elapsed_days'] = (datetime.datetime(2020, 4, 30) - df_all[COL_LAST_REVIEW]).dt.days
df_all.fillna(0, inplace=True)

In [7]:
for col in LIST_LABEL_ENC:
    le = LabelEncoder()
    df_all[col] = le.fit_transform(df_all[col])

In [8]:
df_all = df_all[LIST_USE_COL]

In [9]:
df_all.head()

,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,elapsed_days,reviews_per_month,availability_365
0,9,35.68185,139.80310,0,1,55,5.0,2.21,173
1,21,35.72063,139.78536,0,6,72,36.0,2.11,9
2,7,35.74723,139.82349,0,1,18,38.0,3.46,288
3,16,35.68456,139.68077,0,1,2,28.0,1.76,87
4,18,35.69840,139.70467,0,1,86,91.0,2.00,156


In [10]:
df_all_station_info = pd.concat([df_train_station_info, df_test_station_info], axis=0).reset_index(drop=True)
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA(n_components=10, random_state=0))
])

features_tmp = pipe.fit_transform(df_all_station_info)
df_distance_features = pd.DataFrame(features_tmp, columns=[f'PCA_{i+1}' for i in range(features_tmp.shape[1])])

In [11]:
df_distance_features.head()

,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10
0,18.653366,0.858400,-13.804691,2.301068,-0.302593,2.757417,0.460876,-1.628651,0.657036,-0.117527
1,17.356235,-8.494013,-0.796745,0.504377,-2.052830,-1.859290,0.186437,0.440242,-0.522165,0.054421
2,32.363401,6.726955,13.218806,0.054782,2.093366,-2.422866,-1.568053,0.301075,-0.832967,0.726261
3,-24.277259,0.255721,-0.039130,1.613609,2.273263,-1.378739,0.306705,-0.403338,-0.098069,-0.514598
4,-16.020399,-8.673477,-1.703626,-1.104780,2.383075,-0.055141,1.253202,0.417800,-0.442989,0.166816


In [12]:
df_train_distance_features = df_distance_features[:df_train.shape[0]].reset_index(drop=True)
df_test_distance_features = df_distance_features[df_train.shape[0]:].reset_index(drop=True)

In [13]:
X = df_all[:df_train.shape[0]].reset_index(drop=True)
X = pd.concat([X, df_train_distance_features, df_train_svd_name], axis=1)
y = np.log1p(df_train[COL_Y])

X_inference = df_all[df_train.shape[0]:].reset_index(drop=True)
X_inference = pd.concat([X_inference, df_test_distance_features, df_test_svd_name], axis=1)

In [14]:
X.head()

,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,elapsed_days,reviews_per_month,availability_365,PCA_1,...,name_nmf_41,name_nmf_42,name_nmf_43,name_nmf_44,name_nmf_45,name_nmf_46,name_nmf_47,name_nmf_48,name_nmf_49,name_nmf_50
0,9,35.68185,139.80310,0,1,55,5.0,2.21,173,18.653366,...,0.000000,0.003259,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00057
1,21,35.72063,139.78536,0,6,72,36.0,2.11,9,17.356235,...,0.000000,0.001156,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000
2,7,35.74723,139.82349,0,1,18,38.0,3.46,288,32.363401,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000
3,16,35.68456,139.68077,0,1,2,28.0,1.76,87,-24.277259,...,0.007228,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000
4,18,35.69840,139.70467,0,1,86,91.0,2.00,156,-16.020399,...,0.003686,0.000000,0.0,0.121278,0.0,0.0,0.0,0.0,0.0,0.00000


In [ ]:
# optunaの目的関数を設定する
def objective(trial):
    list_scores = []
    kf = KFold(n_splits=5)
    
    criterion = trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'poisson'])
    max_depth = trial.suggest_int('max_depth', 1, 1000)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt','log2'])
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 1, 1000)
    n_estimators =  trial.suggest_int('n_estimators', 1, 1000)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 5)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    regr = RandomForestRegressor(
                                 criterion=criterion,
                                 max_depth=max_depth,
                                 max_features=max_features,
                                 max_leaf_nodes=max_leaf_nodes,
                                 n_estimators=n_estimators,
                                 min_samples_split=min_samples_split,
                                 min_samples_leaf=min_samples_leaf
                                 )

    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.loc[train_idx, :], X.loc[test_idx, :]
        y_train, y_test = y[train_idx], y[test_idx]
        regr.fit(X_train, y_train)
        pred = regr.predict(X_test)
        score = np.sqrt(mean_squared_error(y_test, pred))
        print(f'score: {score}')
        list_scores.append(score)
    
    rmsle_mean = np.mean(list_scores)
    print(f'rmse_mean: {rmsle_mean}')

    return rmsle_mean

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30, n_jobs=-1)

In [ ]:
study.best_params

{'criterion': 'squared_error',
 'max_depth': 737,
 'max_features': 'sqrt',
 'max_leaf_nodes': 638,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 600}

In [16]:
from numpy.ma.core import argsort
scores = []
preds = []
va_idxes = []
kf = KFold(n_splits=5, shuffle=True, random_state=0)
for i, (train_idx, valid_idx) in enumerate(kf.split(X), start=1):
    print('='*30)
    print(f'fold: {i}')
    X_train, X_valid = X.loc[train_idx, :], X.loc[valid_idx, :]
    y_train, y_valid = y[train_idx], y[valid_idx]
    model = RandomForestRegressor(random_state=0, n_jobs=-1)
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    score = np.sqrt(mean_squared_error(y_valid, pred))
    print(f'score: {score}')
    scores.append(score)
    preds.append(pred)
    va_idxes.append(valid_idx)

    # 構築したモデルの保存
    filename = f'models/rf_fold{i}.pkl'
    pickle.dump(model, open(filename, 'wb'))

va_idxes = np.concatenate(va_idxes)
preds = np.concatenate(preds, axis=0)
order = argsort(va_idxes)
pred_oof = preds[order]
rmsle_mean = np.mean(scores)
print(f'rmse_mean: {rmsle_mean}')

fold: 1
score: 0.6202785185953722
fold: 2
score: 0.5992057651221498
fold: 3
score: 0.5971017659500133
fold: 4
score: 0.5728614874530262
fold: 5
score: 0.5917340994491678
rmse_mean: 0.5962363273139458


In [17]:
df_oof = pd.DataFrame(data=pred_oof, columns=['rf_stacking'])
df_oof.head()

,rf_stacking
0,9.744429
1,9.748376
2,9.691751
3,9.030463
4,9.734504


In [21]:
df_oof.to_csv('input/rf_out_of_fold.csv', index=False)

In [22]:
list_preds_tmp = []

for i in range(1, 6):
    model_path = f'models/rf_fold{i}.pkl'
    model = pickle.load(open(model_path, 'rb'))
    pred = model.predict(X_inference)
    list_preds_tmp.append(pred)

In [23]:
df_preds = pd.DataFrame({'model_1': np.squeeze(list_preds_tmp[0]),
                         'model_2': np.squeeze(list_preds_tmp[1]),
                         'model_3': np.squeeze(list_preds_tmp[2]),
                         'model_4': np.squeeze(list_preds_tmp[3]),
                         'model_5': np.squeeze(list_preds_tmp[4])})

df_preds['pred_avg'] = df_preds.mean(axis=1)

In [24]:
df_preds.head()

,model_1,model_2,model_3,model_4,model_5,pred_avg
0,9.590477,9.511073,9.489887,9.505498,9.414581,9.502303
1,9.723974,9.701156,9.548400,9.430642,9.721432,9.625121
2,9.642879,9.628018,9.623809,9.576232,9.624186,9.619025
3,9.505202,9.597135,9.386064,9.433494,9.416140,9.467607
4,9.600912,9.486341,9.402161,9.461302,9.481619,9.486467


In [25]:
sample_sub[COL_Y] = np.expm1(df_preds['pred_avg'])
sample_sub.head()

,id,y
0,1,13389.533636
1,2,15139.380390
2,3,15047.370157
3,4,12932.900023
4,5,13179.148420


In [26]:
sample_sub.to_csv('submit/submission_rf_1-0-0.csv', index=False)